In [3]:
import pandas as pd

encoded_data = pd.read_csv('encoded_data.csv')
encoded_data.head()

,Customer Name,uid,business start date,business duration (years),Normalized Performance Index,business sector_Agriculture,business sector_Fisheries,business sector_Hospitality,business sector_IT,business sector_Manufacturing,...,education level_Vocational,duration range_0 - 5 years,duration range_5 - 10 years,duration range_10 - 15 years,duration range_15 - 20 years,transaction range_0 - 2000,transaction range_2000 - 4000,transaction range_4000 - 6000,transaction range_6000 - 8000,transaction range_8000 - 11000
0,William Alice,8b37287f-cf0f-4305-97be-cbb21fa68b33,2021-09-15,2.10,0.593091,0,0,0,0,1,...,0,1,0,0,0,1,0,0,0,0
1,Florence Sarah,eb89a5e6-644f-4f10-8886-31b4d37fc6a8,2018-03-16,5.60,0.595990,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
2,James Jane,34d1c009-65a9-44d5-a990-c8b4c5e7fb9f,2013-06-24,10.33,0.560126,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,Nakinobe Jane,72db3880-aab0-4782-bc73-a47f787d30d1,2018-04-24,5.49,0.590698,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
4,Bridget Jane,57493f5c-80a0-4a41-9381-3d9241de74f4,2010-03-18,13.60,0.495512,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0


## Use Kmeans Clusters to create clusters

In [4]:
from sklearn.cluster import KMeans

encoded_data_cleaned = encoded_data.drop(columns=['Customer Name', 'uid', 'business start date'])

# apply kmeans
kmeans = KMeans(n_clusters=5, random_state=42)

#create a new column called cluster and add the kmeans clusters into that column
encoded_data_cleaned['cluster'] = kmeans.fit_predict(encoded_data_cleaned) 

# update the orgiginal clustering_data_df with the cluster column since we need the originial df to perfom the merge with the customer_data
# (need the uid)
encoded_data['cluster'] = encoded_data_cleaned['cluster'] 

encoded_data.head(3)

/Users/power/Powem/pomucapital/pomu-data-analysis/venvpomu/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,Customer Name,uid,business start date,business duration (years),Normalized Performance Index,business sector_Agriculture,business sector_Fisheries,business sector_Hospitality,business sector_IT,business sector_Manufacturing,...,duration range_0 - 5 years,duration range_5 - 10 years,duration range_10 - 15 years,duration range_15 - 20 years,transaction range_0 - 2000,transaction range_2000 - 4000,transaction range_4000 - 6000,transaction range_6000 - 8000,transaction range_8000 - 11000,cluster
0,William Alice,8b37287f-cf0f-4305-97be-cbb21fa68b33,2021-09-15,2.10,0.593091,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
1,Florence Sarah,eb89a5e6-644f-4f10-8886-31b4d37fc6a8,2018-03-16,5.60,0.595990,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,3
2,James Jane,34d1c009-65a9-44d5-a990-c8b4c5e7fb9f,2013-06-24,10.33,0.560126,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,4


## Cluster Analysis
Given the 5 clusters we have created, we can now start analysis to answer our business question. 
ToDo later
